# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Trusting the compiler”

This lesson is more of a conversation about _type inference_. Isaac makes the following points:

- the C# `var` keyword implements compile-time, local-scope type inference while the `dynamic` keyword makes similar attempts at runtime and has nothing to do with F#
- F# type inference is based on [the Hindley-Milner (HM) type system](https://en.wikipedia.org/wiki/Hindley%E2%80%93Milner_type_system)
- because F# type inference can “escape local scope,” refactoring is made easier
- type inference may not work well with a type from a C# library
- in F#, implicit conversions are not allowed

The following invalid C# dramatizes what F# is capable of:


In [ ]:
public static var CloseAccount(var balance, var customer)
{
    if(balance < 0) return false;
    else
    {
        customer.Withdraw(customer.AccountBalance);
        customer.Close();
        return true;
    }
}

Error: (1,32): error CS0825: The contextual keyword 'var' may only appear within a local variable declaration or in script code
(1,45): error CS0825: The contextual keyword 'var' may only appear within a local variable declaration or in script code
(1,15): error CS0825: The contextual keyword 'var' may only appear within a local variable declaration or in script code

In [ ]:
let closeAccount(balance, customer) =
    if balance < 0 then false
    else
        customer.Withdraw customer.AccountBalance
        customer.Close()
        true

Error: input.fsx (4,9)-(4,26) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (5,9)-(5,23) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.

Even though the F# error message above is much longer, notice how the type `balance` is inferred as `int` (but it probably should be `decimal`). On the other extreme, `customer` cannot be inferred at all and looks like a type that is saturated in OOP complexity:

- `customer` looks like it has a function (method) `Withdraw` that looks like it has one (probably `decimal`) argument
- `customer` looks like it has an `AccountBalance` member (property or field) that is inelegantly being passed to its own function
- `customer` looks like it has a `Close` function that takes no arguments

We can skip ahead in the book (again) to deal with this foreign-looking `customer` (OOP) object and define an F# interface to help the inference engine:


In [ ]:
type ICustomer =
    abstract member AccountBalance : decimal
    abstract member Close : unit -> unit
    abstract member Withdraw : decimal -> unit


We will depend on Isaac to explain to us later how we are defining `ICustomer`. One important detail to mention now is that `unit` in F# is kind of like `void` in C-like languages [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/unit-type)].

We use F# type notation (which resembles Typescript type notation) to explicitly specify the type of `customer`:

In [ ]:
let closeAccount(balance, customer: ICustomer) =
    if balance < 0m then false
    else
        customer.Withdraw customer.AccountBalance
        customer.Close()
        true

Also take note of the `decimal` literal, `0m`, added above. This allows the inference engine to assert that `balance` is `decimal` instead of `int`.


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.235701+3881a96164de75fca84f5f11027f3606b7878044Build date: 2021-07-11T04:06:39.6100964Zhttps://github.com/dotnet/interactive
